# LLM Finetuning Strategies

Fine-tuning is to take an existing model architecture and adapt its weights to learn context specific to a particular domain.

## Common Use-Cases

- **Language Learning**: allows the model to learn and recognize patterns in the new language. In contrast, simple RAG applications do not effectively connect new contexts with existing knowledge.
- **Safeguarding LLMs Effectively**: prevent harmful responses coming from LLMs such as mental health.
- **AI Personas**: make snippets from various sources align with your specific style or standards.
- **Smarter, Smaller Models**: reduces the costs while the models are still efficient and cost-effective for specific needs.

## Fine-Tuning Process

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/0*NWzpXGgfgJRhcYcG' width=700>

The 3 popular **approaches** for fine-tuning LLMs:

1. **Supervised Learning**: The model learns through training on labeled input-output pairs. This approach, like instruction fine-tuning, refines the model's performance on specific tasks by providing correct examples and feedback, enhancing its accuracy in responding to particular instructions.

2. **Self-Supervised Learning**: The model leverages patterns within the data itself, without needing labeled examples, to make predictions about missing or hidden parts of the data. This method improves the model's understanding of language through methods like [masked language modeling](https://huggingface.co/docs/transformers/main/en/tasks/masked_language_modeling) (BERT), [autoregressive modeling](https://aws.amazon.com/what-is/autoregressive-models/) (GPT), and [contrastive learning](https://encord.com/blog/guide-to-contrastive-learning/) (SimCLR), [Next Sentence Prediction](https://www.youtube.com/watch?v=1gN1snKBLP0) (BERT) and Permutation Language Modeling ([XLNet](https://arxiv.org/pdf/1906.08237)).

3. **Reinforcement Learning (RL)**: The model is trained to optimize responses through a reward-based system. High-quality outputs receive rewards, while poor ones get penalties. Over time, this feedback helps the model refine its responses, guided by various policy optimization strategies to align closely with desired human-like outputs.

The 2 fine-tuning **strategies**:
- **Horizontal fine-tuning**:
    - adapting a model to perform well across a range of similar tasks or domains.
    - The model is finetuned on data that spans multiple related areas without specializing in any single one.
    - Can handle various tasks while retaining the base model’s generalist nature.

- **Vertical fine-tuning**
    - adapting a model to excel in a specific task or domain.
    - The model is finetuned using highly specialized or domain-specific data
    - enabling model to better understand and generate highly accurate responses in particular area.

It’s also crucial to consider:
- the required number of parameters
- the computational resources needed.
- the specific task and data type. 
- choosing a top-down or bottom-up approach
- training on individual or batched layers
- appropriate fine-tuning modules to create an effective infrastructure. 

For practical implementation, parameter-efficient vertical fine-tuning is emphasized.

The 3 strategies for fine-tuning include:
- full parameter retraining
- Parameter-Efficient Fine-Tuning (PEFT)
- transfer learning

## PEFT

Parameter-Efficient Fine-Tuning (PEFT) reduces the need to update all parameters in a large language model, focusing instead on a small subset. By freezing most parameters, maintaining residual connections, and using regularization, PEFT preserves the model’s existing knowledge and prevents [catastrophic forgetting](https://cobusgreyling.medium.com/catastrophic-forgetting-in-llms-bf345760e6e2). Techniques like [GaLore](https://github.com/jiaweizzhao/GaLore) even enable fine-tuning large models like Llama-3 on personal computers, making advanced language modeling more accessible.

### Techniques for PEFT

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*aj45RItrFN5lEk69Owchzw.png' width=550>

PEFT methods are classified under 3 broad categories:
- **Addition**-based (adding new trainable parameters).
- **Selection**-based (selecting a subset of parameters from the base model)
- **Reparametrization**-based (having a alternate representation).

#### [Adapters](https://github.com/AGI-Edgerunners/LLM-Adapters)

Adapters, part of the **addition** class, are lightweight feed-forward modules inserted between FC layers in transformer models to reduce parameter space. 

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*1NFDivOl7qoC9coZVTqthA.png' width=600>

By downscaling and then upscaling dimensions, they use far fewer parameters than FC layers. For instance, reducing a 256-dimensional input to 16 dimensions and then back requires 8,192 parameters, versus 65,536 for a full 256-to-256 mapping. Adapter tuning only trains adapters, layer norms, and the final head, leading to faster, efficient tuning. This approach allows BERT to achieve comparable performance to full fine-tuning with just 3.6% of parameters trained.

In [1]:
from torch import nn
class AdapterBlock(nn.Module):
    def __init__(self, input_dim, adapter_dim=64):
        super(AdapterBlock, self).__init__()
        self.down_proj = nn.Linear(input_dim, adapter_dim)
        self.activation = nn.ReLU()
        self.up_proj = nn.Linear(adapter_dim, input_dim)
    def forward(self, x):
        return self.up_proj(self.activation(self.down_proj(x)))

A specialized adapter, [Llama-Adapter](https://github.com/OpenGVLab/LLaMA-Adapter), tailors Llama models for instruction-following.

#### [Prompt Tuning](https://github.com/mkshing/Prompt-Tuning)

Prompt tuning is a **additive** PEFT method that uses "soft prompts" — dynamically updated prompts based on loss feedback — instead of static, human-created prompts. Unlike prompt engineering, which can be labor-intensive and limited by context window length, prompt tuning optimizes model responses by adding trainable tokens to the input embedding without changing model weights. These tokens, representing any word in embedding space, adapt to find the best representation for guiding model output. 

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*gxVgfrVAGTZLIYKgsNVTOg.png' width=400> 

Typically, tuning 20 tokens suffices for classification tasks, reducing parameter work to just 20K instead of billions. This method performs well on larger models but is less effective on smaller ones and challenging to control due to its "black-box" nature.

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*L9uSM5ubQC8iEEK3fMQFIg.png' width=400>

In [ ]:
import torch
from torch import nn
class PromptTuningBlock(nn.Module):
    def __init__(self, input_dim, prompt_length, hidden_dim, output_dim):
        super(PromptTuningBlock, self).__init__()
        self.prompt_length = prompt_length
        self.prompt_embeddings = nn.Parameter(torch.randn(prompt_length, input_dim))
        self.input_embedding = nn.EmbeddingBag(num_embeddings=input_dim, embedding_dim=hidden_dim)
        # A simple linear layer for output
        self.fc = nn.Linear(hidden_dim + prompt_length * input_dim, output_dim)
    def forward(self, input_ids):
        # Get input embeddings
        input_embeds = self.input_embedding(input_ids)
        # Concatenate prompt embeddings
        combined_embeds = torch.cat([input_embeds, self.prompt_embeddings.mean(dim=0).unsqueeze(0)], dim=1)
        # Pass through the linear layer
        output = self.fc(combined_embeds)
        return output

In [ ]:
input_dim = 768    # The dimensionality of each input token (e.g., token embedding size in BERT-like model)
prompt_length = 20 # Number of trainable prompt tokens
hidden_dim = 512   # Dimension of intermediate representation
output_dim = 10    # Output dimension (e.g., number of classes for classification)

model = PromptTuningBlock(input_dim, prompt_length, hidden_dim, output_dim)

# Create dummy input data
batch_size = 4     # Number of samples in batch
seq_length = 10    # Length of input sequence
input_ids = torch.randint(low=0, high=input_dim, size=(batch_size, seq_length))  # Random token indices
print(input_ids)

output = model(input_ids)
print(output.shape)  # Expected: [batch_size, output_dim]

#### [Prefix Tuning](https://github.com/XiangLi1999/PrefixTuning)

Prefix tuning is an advanced form of **prompt tuning** that introduces trainable "soft prompts" at each transformer block, applied before positional encoding. This technique differs from prompt tuning by expanding influence across multiple model layers rather than just modifying the input embeddings. While prefix tuning adjusts more parameters for improved task-specific performance, it may increase computational demand and the risk of overfitting. Conversely, prompt tuning is more parameter-efficient but could be less adaptable to complex tasks due to fewer updated parameters.

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*VDMUtRyn5kEQaG8oxuOahg.png' width=500>

In [20]:
import torch
from torch import nn
class PrefixTuningBlock(nn.Module):
    def __init__(self, num_prefix_tokens, hidden_size, num_layers):
        super(PrefixTuningBlock, self).__init__()
        self.prefix_tokens = nn.Parameter(torch.randn(num_prefix_tokens, hidden_size))
        self.transformer_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=hidden_size, nhead=8, dropout=0.1) for _ in range(num_layers)
        ])
    def forward(self, input_ids):
        # Create a tensor of prefix tokens
        prefix_tokens = self.prefix_tokens.unsqueeze(0).expand(input_ids.size(0), -1, -1)
        # Concatenate input_ids and prefix_tokens
        input_ids = torch.cat([prefix_tokens, input_ids], dim=1)
        # Pass the concatenated input through the transformer layers
        output = input_ids
        for layer in self.transformer_layers:
            output = layer(output)
        return output

In [21]:
num_prefix_tokens = 10    # Number of prefix tokens
hidden_size = 512         # Dimension of hidden layer in the transformer
num_layers = 6            # Number of transformer layers
batch_size = 2            # Number of sequences in a batch
seq_len = 20              # Length of each input sequence

# Create an instance of PrefixTuningBlock
prefix_tuning_block = PrefixTuningBlock(num_prefix_tokens=num_prefix_tokens, 
                                        hidden_size=hidden_size, 
                                        num_layers=num_layers)

# Dummy input tensor of shape (batch_size, seq_len, hidden_size)
input_ids = torch.randn(batch_size, seq_len, hidden_size)
print(input_ids)

output = prefix_tuning_block(input_ids)

# Output shape should be (batch_size, seq_len + num_prefix_tokens, hidden_size)
print("Output shape:", output.shape)

tensor([[[-0.4692, -0.6542, -1.6399,  ..., -0.9354,  0.0892,  0.6288],
         [-1.4637,  1.2387, -1.0630,  ...,  0.5636,  0.0446,  2.8761],
         [ 0.3037, -0.5612, -1.7132,  ...,  0.8258,  0.7978,  0.7917],
         ...,
         [ 1.4325, -2.1468,  0.1886,  ..., -0.7694,  0.3839, -1.2466],
         [ 1.5146,  0.1461,  0.8084,  ..., -0.0236,  0.8259, -0.5386],
         [ 0.2051, -0.8629,  1.1774,  ...,  0.3925,  0.2329,  2.2460]],

        [[ 0.4043, -0.8197,  0.9327,  ...,  0.4811,  0.5672,  0.9318],
         [ 1.6717, -1.7679,  0.1913,  ...,  0.2734, -2.9206,  0.8655],
         [ 0.7312,  2.0188, -0.4299,  ..., -0.6321, -0.4581,  0.2430],
         ...,
         [-0.0391, -1.2895, -1.3723,  ...,  0.2548,  0.9435, -0.8906],
         [-1.5573,  0.6870, -2.0728,  ...,  0.1486, -0.9864, -0.5716],
         [ 0.1022,  0.6183, -1.1393,  ..., -0.9104, -0.6915, -0.4101]]])
Output shape: torch.Size([2, 30, 512])


#### [Low-Rank Adaptation of LLMs](https://github.com/microsoft/LoRA) (LoRA Family)

This **reparameterization** method reduces the fine-tuning parameter count by breaking down a large weight matrix in attention layers into two smaller matrices, using an alternative representation called pseudo decomposition. Instead of directly decomposing the matrix, it learns through this reduced representation, adjusting fewer parameters. The rank \(r\) of the decomposed matrix is typically scaled to the training data size and model size to prevent overfitting and manage computational costs. LoRA's approach allows it to learn minimally and retain prior knowledge effectively, leading to less forgetting and balanced fine-tuning.

The decomposition:

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/0*sn_RqGAV45Newo3k.png' width=400>

Given 2 matrices A and B with 100 and 500 parameters.

=> Total number of parameters in $ΔW$ is $100\times500=50000$.

Assume a rank of 5

=> The new weight matrices $W_A$ and $W_B$ become $100\times5=500$ and $500\times5=2500$

=> The new $ΔW=W_A+W_B=500+2500=3000$ parameters (a 94\% decrease).

In [2]:
import torch
import torch.nn as nn
import math

class LoRA(nn.Module):
    def __init__(self, input_dim, output_dim, rank=8, alpha=1.0):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.rank = rank
        self.alpha = alpha
        # Create LoRA weight matrices
        self.W_A = nn.Parameter(torch.empty(input_dim, rank))
        self.W_B = nn.Parameter(torch.empty(rank, output_dim))
        # Initialize LoRA weights
        nn.init.kaiming_uniform_(self.W_A, a=math.sqrt(5))
        nn.init.zeros_(self.W_B)

    def forward(self, x, W):
        h = x @ W
        # Low-Rank Update
        h += self.alpha * x @ (self.W_A @ self.W_B)
        return h

In [4]:
# Define dimensions
input_dim = 128  # e.g., input feature size
output_dim = 256  # e.g., output feature size

# Initialize LoRA module
lora = LoRA(input_dim=input_dim, output_dim=output_dim, rank=8, alpha=1.0)

# Create a dummy input tensor
batch_size = 32
x = torch.randn(batch_size, input_dim)

# Create a weight matrix
W = torch.randn(input_dim, output_dim)

# Perform the forward pass
output = lora(x, W)
output

tensor([[  3.9933, -15.8321, -21.2333,  ...,  -6.0150,  -4.2069, -12.1648],
        [ -9.7652,   3.5665,   4.1797,  ...,  32.3229,  -2.1751,   1.8548],
        [-17.7824,  10.0562, -30.9159,  ...,  -1.7471,  -2.5186, -15.6680],
        ...,
        [ -0.7930,  -5.7677,  18.4439,  ...,   5.9187,  -5.2896,  -3.1255],
        [  7.5565,  -6.5402, -19.5440,  ...,  -7.4357,   1.4648,  -6.9477],
        [ -7.8752,   6.1167,  12.9737,  ...,   6.0308,  -9.1869,  -9.8675]],
       grad_fn=<AddBackward0>)

In [5]:
## Incorporate LoRA in a Larger Model
class ModelWithLoRA(nn.Module):
    def __init__(self, input_dim, output_dim, rank=8, alpha=1.0):
        super().__init__()
        self.base_layer = nn.Linear(input_dim, output_dim)
        self.lora = LoRA(input_dim, output_dim, rank, alpha)

    def forward(self, x):
        W = self.base_layer.weight.t()  # Get the base layer weight
        return self.lora(x, W)

# Instantiate the model
model = ModelWithLoRA(input_dim, output_dim)
output = model(x)
output

tensor([[ 0.7932,  0.1635, -0.1747,  ..., -0.1391,  0.9054, -0.4191],
        [ 0.7876, -0.1774,  1.2132,  ..., -0.4322,  0.2714,  0.3598],
        [-0.0166, -0.6446, -0.2640,  ...,  0.3267, -0.1368, -1.5596],
        ...,
        [ 0.3955,  0.8044,  0.4201,  ...,  0.3359, -0.3672,  0.3622],
        [-0.1300,  0.1261,  0.7624,  ...,  0.2812, -0.1955, -0.1954],
        [ 0.0660, -0.5791, -0.1490,  ...,  0.2672, -0.7398, -0.0021]],
       grad_fn=<AddBackward0>)

There are various [flavors](https://gautam75.medium.com/exploring-different-lora-variants-for-efficient-llm-fine-tuning-4ca41179e658#:~:text=In%20the%20world%20of%20Generative,-Rank%20Adaptation%20(LoRA)) of LoRA like DoRA, QLoRA (a popular mixed precision strategy) , LoHA, etc.

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/0*e0JdKfE9rlFAL0qM.jpg' width=500>

#### [Infused Adapter by Inhibiting and Amplifying Inner Activations $(IA)^3$](https://github.com/r-three/t-few)

This method is an **additive** approach that involves 3 phases: adding vectors, rescaling, and fine-tuning. It introduces 3 learned vectors:

1. **Key Rescaling Vector**: Multiplied with the keys in the self-attention layer.
2. **Value Rescaling Vector**: Multiplied with the values in the self-attention and encoder-decoder attention layers.
3. **Intermediate Activation Rescaling Vector**: Multiplied with the intermediate activations in the feed-forward network.

These vectors are used to either inhibit (reduce) or amplify (increase) the corresponding activations in the model. After this rescaling, the model is fine-tuned on a downstream task, and the learned vectors are updated to optimize performance. This method is considered a better alternative to few-shot prompting (in-context learning).

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*2o8fa5svXMi0KWkZLc1kTQ.png' width=700>

#### Orthogonal Finetuning via Butterfly Factorization (BOFT)

This **reparameterization** strategy involves performing an orthogonal transformation of weight matrices using butterfly factorization. Butterfly factorization represents a weight matrix as the product of a diagonal matrix and a permutation matrix. The goal is to maintain structural constraints by introducing orthogonality between the gradients of fine-tuning and pre-training losses, ensuring the preservation of hyperspherical energy. This energy represents the distance of a point from the origin on a hypersphere, and maintaining it during fine-tuning helps retain learned information, reducing the risk of forgetting. The sparse representation enabled by this method aids in better model generalization. The butterfly structure also provides flexibility and parameter efficiency, making it suitable for image models and text-to-image models.

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*q9aMQ5LIVCV4onTAooeyAw.png' width=500>

Alternatives to this approach include sharing low-rank matrices across layers and initializing adapter layers with the singular values and vectors of the original model’s weights. This method is particularly beneficial for training models on local hardware, such as smartphones and PCs, where computational resources are limited.

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/0*-dZXwll4srkJRX63.png' width=500>

# Reference

[LLM Finetuning Strategies](https://pub.towardsai.net/llm-finetuning-strategies-f1e2e8d91b30)